In [1]:
# # This cell runs bowtie locally, uncomment last line to run the command and test your bowtie
# 
# import os
# # wget index and SRR043378 directly into project folder
# threads = 4  # Can be changed
# # folder name then file name
# bowtieIndex = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
#               "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"
# # fastq read file


# reads = "SRR043378_1.fastq/data"
# cmd = "bowtie2 -p "+str(threads)+" -x " + bowtieIndex + " -U "+reads+" -S out.sam"
# 
# 
# os.system(cmd)

In [2]:
# import pyspark
# import os
# 
# # new sparkContext
# sc = pyspark.SparkContext(appName="DNA")

In [3]:
# Basis for map function

# threads = 1  # Can be changed
# # folder name then file name
# bowtieIndex = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
#               "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"
# # fastq read file
# reads = "SRR043378_1.fastq/data"
# cmd = "bowtie2 -p " + str(threads) + " -x " + bowtieIndex + " -U " + reads + " -S out.sam"
# os.system(cmd)

In [1]:
#import findspark
#findspark.init('/home/taylor/spark')

from pyspark import SparkConf, SparkContext, TaskContext
import math
from pyspark.sql import SQLContext


conf = SparkConf().setMaster("local").setAppName("DNA")

sc = SparkContext(conf=conf)
# input_file = "SRR043378_1.fastq/data"
input_file = "yeast_40K_reads.fq"

In [2]:
# creates tuples with format (string, line number)
raw_input = (sc.textFile(input_file)).zipWithIndex()

In [3]:
# create tuple with format (read number, string)
indexed_raw_input = raw_input.map(lambda x: (math.floor(x[1]/4), x[0]))

In [4]:
# Function to pass to map function that concatenates the 4 strings
def make_reads(iterable_read):
    x = 0
    line1 = ""
    line2 = ""
    line3 = ""
    line4 = ""
    for line in iterable_read:
        if x == 0:
            line1 = line
        if x == 1:
            line2 = line
        if x == 2:
            line3 = line
        if x == 3:
            line4 = line
        x = x+1
    return "%s\n%s\n%s\n%s\n" % (line1, line2, line3, line4)

In [5]:
reads_tuple = indexed_raw_input.groupByKey().mapValues(lambda x: make_reads(x))

In [6]:
# Sort by key, which is just the line number, and then get the values, which is just the read.
readsRDD = reads_tuple.sortByKey().values()

In [7]:
# test = reads_tuple.take(10)
# Test get one record (brackets denote first tuple, second field because it is in format (0, read), (1, read) etc...)
# print(test[0][1])

In [8]:
# num_reads = reads_tuple.count()

In [9]:
# print("Number of reads: %s" % num_reads)
# 16,924,141 reads

In [10]:
# test2 = readsRDD.take(10)

In [11]:
# print(test2[0])

In [12]:
# readsRDD = readsRDD.repartition(10)

In [13]:
# readsRDD.mapPartitionsWithIndex()

In [14]:
# readsRDD.saveAsTextFile('partitioned_data/')

In [15]:
# bowtie_index = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
#                "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"

bowtie_index = "Bowtie2Index/genome"
# try:
alignment_pipe = readsRDD.pipe("bowtie2 --no-hd --no-sq -x " + bowtie_index + " -")
# except Exception as ex:
#     print(ex)

In [16]:
import os


def reduce_to_sam(output):
    try:
        if not os.path.exists("output"):
            os.makedirs("output")
        ctx = TaskContext()
        _id = str(ctx.partitionId())
        file = open("output/output_"+_id+".sam", "a+")
        # print(type(output))
        # tempRDD = output.collect()
        # tempRDD.saveAsTextFile(str(TaskContext().partitionId()) + 'output.sam')
        file.write(output+'\n')
        file.close()
    except Exception as ex:
        print(ex)

In [26]:
# file = open("output.sam", 'r+b')
# alignment_pipe.foreach(lambda x: reduce_to_sam(x))  # .saveAsTextFile('output.sam')  # .saveAsTextFile('output.sam') 
output = alignment_pipe.mapPartitionsWithIndex(lambda x,y: x+y)
# file.close()

In [27]:
# alignment_pipe.foreach(lambda x: print(x))
test3 = output.take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 6, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/taylor/PycharmProjects/DNA_Project/venv/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/home/taylor/PycharmProjects/DNA_Project/venv/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/taylor/PycharmProjects/DNA_Project/venv/lib/python3.6/site-packages/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/taylor/PycharmProjects/DNA_Project/venv/lib/python3.6/sit

In [ ]:
# TODO: split data into batches and send to bowtie 
# https://github.com/citiususc/SparkBWA/blob/master/src/main/java/com/github/sparkbwa/BwaAlignmentBase.java


In [19]:
sc.stop()